# 📘 Laporan Proyek: Sistem Rekomendasi Buku

---
By : Rahmi Amilia


Proyek ini bertujuan membangun sistem rekomendasi buku berbasis dua pendekatan: Collaborative Filtering (SVD) dan Content-Based Filtering. Dataset yang digunakan adalah Book-Crossing Dataset yang terdiri dari informasi buku, pengguna, dan rating.

Notebook ini mencakup tahapan: import library, load data, data understanding, data preparation, modeling, dan evaluasi model.

### 1. Import Library

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings("ignore")

### 2. Load Dataset

Memuat data dari file CSV Book-Crossing ke dalam DataFrame: books, ratings, dan users.

In [89]:
books = pd.read_csv('/content/BX_Books.csv', sep=';', encoding='latin-1', nrows=10000)
ratings = pd.read_csv("/content/BX-Book-Ratings.csv", sep=';', encoding='latin-1', on_bad_lines='skip')
users = pd.read_csv("/content/BX-Users.csv", sep=';', encoding='latin-1', on_bad_lines='skip')

### 3. Data Understanding

Dataset terdiri dari:
- books.csv: 10.000 baris, 8 kolom
- ratings.csv: 1.149.780 baris, 3 kolom
- users.csv: 278.858 baris, 3 kolom

Fitur penting:
- `isbn`, `title`, `author`, `year`, `publisher`, `image_s/m/l`
- `user_id`, `location`, `age`
- `rating` (0-10)

> Kolom `age` memiliki missing values dan outlier seperti usia <5 atau >100.

Berikut adalah jumlah baris dan kolom pada masing-masing dataset sebelum dilakukan proses pembersihan:


In [90]:
print('jumlah data books : ', books.shape)
print('jumlah data ratings : ', ratings.shape)
print('jumlah data users : ', users.shape)

jumlah data books :  (10000, 8)
jumlah data ratings :  (1149780, 3)
jumlah data users :  (278858, 3)


Dataset `ratings` terdiri dari 3 kolom utama:

- **user_id**: ID unik dari pengguna yang memberikan rating
- **isbn**: ID unik dari buku yang diberi rating
- **rating**: Nilai rating yang diberikan oleh pengguna terhadap buku (skala 0–10)

In [91]:
ratings.columns = ["user_id", "isbn", "rating"]
ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Dataset `books` memiliki 8 kolom, yaitu:

- **isbn**: ID unik buku (International Standard Book Number)
- **title**: Judul buku
- **author**: Nama penulis buku
- **year**: Tahun terbit buku
- **publisher**: Nama penerbit
- **image_s**, **image_m**, **image_l**: URL gambar sampul buku dalam ukuran kecil, sedang, dan besar


In [92]:
books.columns = ["isbn", "title", "author", "year", "publisher", "image_s", "image_m", "image_l"]
books.head()

,isbn,title,author,year,publisher,image_s,image_m,image_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


Dataset `users` terdiri dari 3 kolom utama:

- **user_id**: ID unik dari masing-masing pengguna
- **location**: Lokasi pengguna, biasanya berupa format "kota, negara bagian/provinsi, negara"
- **age**: Usia pengguna (dalam tahun), namun mengandung banyak nilai yang hilang (NaN)


In [93]:
users.columns = ["user_id", "location", "age"]
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


### Informasi Struktur dan Statistik Dataset Books

#### a. Struktur Data (`books.info()`)

- Dataset `books` memiliki **10.000 baris** dan **8 kolom**.
- Semua kolom tidak memiliki nilai null (non-null count = 10000).
- Tipe data:
  - 1 kolom bertipe numerik (`year`)
  - 7 kolom bertipe objek (string), termasuk `isbn`, `title`, `author`, dan kolom gambar

#### b. Statistik Deskriptif (`books.describe(include='all')`)

- Kolom **`title`** memiliki 9.553 nilai unik → artinya ada beberapa judul yang berulang.
- Kolom **`author`** memiliki 5.754 nama unik, dengan penulis paling sering muncul adalah **Stephen King** sebanyak **68 kali**.
- Kolom **`publisher`** memiliki 1.701 penerbit unik, dan yang paling sering muncul adalah **Ballantine Books**.
- Kolom **`year`** memiliki nilai minimum 0 dan maksimum 2005, dengan rata-rata sekitar tahun **1958**. Nilai tahun 0 merupakan outlier dan akan difilter pada tahap data preparation.
- Kolom gambar (`image_s`, `image_m`, `image_l`) hampir seluruhnya unik, menunjukkan setiap buku memiliki URL gambar tersendiri.


In [94]:
books.info()
books.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   isbn       10000 non-null  object
 1   title      10000 non-null  object
 2   author     10000 non-null  object
 3   year       10000 non-null  int64 
 4   publisher  10000 non-null  object
 5   image_s    10000 non-null  object
 6   image_m    10000 non-null  object
 7   image_l    10000 non-null  object
dtypes: int64(1), object(7)
memory usage: 625.1+ KB


,isbn,title,author,year,publisher,image_s,image_m,image_l
count,10000,10000,10000,10000.000000,10000,10000,10000,10000
unique,10000,9553,5754,NaN,1701,9999,9999,9999
top,0195153448,"The Golden Compass (His Dark Materials, Book 1)",Stephen King,NaN,Ballantine Books,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
freq,1,5,68,NaN,300,2,2,2
mean,NaN,NaN,NaN,1958.671900,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,269.017284,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,1992.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,1997.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,2001.000000,NaN,NaN,NaN,NaN


In [95]:
users.info()
users.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   278858 non-null  int64  
 1   location  278858 non-null  object 
 2   age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


,user_id,age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


Kolom-kolom pada dataset ratings:

* user_id : ID pengguna.

* isbn : ID buku.

* rating : Nilai rating dari pengguna terhadap buku. Nilai 0 menunjukkan rating implisit (belum memberikan penilaian eksplisit).

In [96]:
ratings.info()
ratings['rating'].value_counts().sort_index()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  1149780 non-null  int64 
 1   isbn     1149780 non-null  object
 2   rating   1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


,count
rating,
0,716109
1,1770
2,2759
3,5996
4,8904
5,50974
6,36924
7,76457
8,103736


### Cek Missing Values dan Statistik Usia

In [97]:
books.columns = ['isbn', 'title', 'author', 'year', 'publisher', 'image_s', 'image_m', 'image_l']
ratings.columns = ['user_id', 'isbn', 'rating']
users.columns = ['user_id', 'location', 'age']
print(users.isnull().sum())
print(users['age'].describe())

user_id          0
location         0
age         110762
dtype: int64
count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: age, dtype: float64


### 4. Data Preparation

### 4.1 Rename Kolom
4.2 Hapus Duplikat ISBN

In [98]:
books.drop_duplicates(subset='isbn', keep='first', inplace=True)

4.3 Filter Buku dengan Tahun Tidak Masuk Akal

In [99]:
books = books[(books['year'] >= 1900) & (books['year'] <= 2025)]

4.4 Hapus Kolom Gambar

In [100]:
books.drop(columns=['image_s', 'image_m', 'image_l'], inplace=True)

4.5 Filter Rating Eksplisit

In [101]:
ratings = ratings[ratings['rating'] > 0]

e. Isi nilai kosong dan gabungkan fitur

In [102]:
books['title'] = books['title'].fillna('')
books['author'] = books['author'].fillna('')
books['combined_features'] = books['title'] + ' ' + books['author']

f. TF-IDF Vectorization untuk Content-Based Filtering

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books['combined_features'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### 5. Modeling & Results

5.1 Collaborative Filtering (SVD)

* Menggunakan Surprise library untuk menerapkan SVD.

* Melatih model menggunakan data eksplisit.

* Memberikan rekomendasi Top-N buku untuk user tertentu.

In [104]:
!pip install scikit-surprise --no-binary scikit-surprise

In [105]:
!pip install numpy==1.23.5

In [106]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings[['user_id', 'isbn', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

svd_model = SVD()
svd_model.fit(trainset)

predictions = svd_model.test(testset)
rmse(predictions)

RMSE: 1.6399


1.6398707223136224

5.2 Contoh Output Rekomendasi SVD

Membuat fungsi rekomendasi untuk user tertentu:

In [107]:
def recommend_books_svd(user_id, books_df, ratings_df, model, n=5):
    user_books = ratings_df[ratings_df['user_id'] == user_id]['isbn'].tolist()
    unread_books = books_df[~books_df['isbn'].isin(user_books)]

    predictions = []
    for isbn in unread_books['isbn']:
        pred = model.predict(user_id, isbn)
        predictions.append((isbn, pred.est))

    top_n = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
    return books_df[books_df['isbn'].isin([isbn for isbn, _ in top_n])][['title', 'author']]

In [108]:
recommend_books_svd(user_id=276729, books_df=books, ratings_df=ratings, model=svd_model)

,title,author
408,The Lovely Bones: A Novel,Alice Sebold
1387,A Prayer for Owen Meany,John Irving
4206,"The Return of the King (The Lord of the Rings,...",J.R.R. TOLKIEN
6330,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling
7994,Voyager,DIANA GABALDON


5.3 Content-Based Filtering

* Gunakan fitur title dan author dari buku.

* Ubah menjadi vektor TF-IDF.

* Hitung Cosine Similarity antar buku.

* Berikan rekomendasi untuk buku tertentu.

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

books['title'] = books['title'].fillna('')
books['author'] = books['author'].fillna('')

books['combined_features'] = books['title'] + ' ' + books['author']

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books['combined_features'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Contoh Output Rekomendasi Content-Based

In [110]:
def get_recommendations(title, books_df, cosine_sim, n=5):
    from difflib import get_close_matches
    import numpy as np
    import pandas as pd
    indices = pd.Series(books_df.index, index=books_df['title']).drop_duplicates()
    if title not in indices:
        closest = get_close_matches(title, books_df['title'], n=1)
        if not closest:
            return "Judul tidak ditemukan, dan tidak ada kemiripan yang cukup."
        else:
            title = closest[0]
            print(f"Menampilkan hasil rekomendasi untuk judul terdekat: '{title}'")
    idx = indices[title]
    sim_vector = cosine_sim[idx]
    if hasattr(sim_vector, 'toarray'):
        sim_vector = sim_vector.toarray().flatten()
    else:
        sim_vector = np.array(sim_vector).flatten()
    sim_scores = list(enumerate(sim_vector))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+10]
    book_indices = [i[0] for i in sim_scores if i[0] < len(books_df)]
    return books_df.iloc[book_indices[:n]][['title', 'author']]

get_recommendations("Harry Potter and the Chamber of Secrets (Book 2)", books, cosine_sim)

,title,author
3515,El Vendedor De Noticias (Espasa Juvenil),Jose Luis Olaizola
2097,"Vieja Sirena, La",Jose Luis Sampedro


In [111]:
from difflib import get_close_matches
get_close_matches("Harry Potter and the Chamber of Secrets", books['title'], n=5)

['Harry Potter and the Chamber of Secrets (Book 2)',
 'Harry Potter and the Chamber of Secrets (Book 2)',
 'Harry Potter and the Goblet of Fire (Book 4)',
 'Harry Potter and the Goblet of Fire (Book 4)',
 'Harry Potter and the Order of the Phoenix (Book 5)']

### 6. Evaluation

6.1 Evaluasi Collaborative Filtering (SVD)

Metrik: RMSE (Root Mean Squared Error)
RMSE mengukur seberapa jauh hasil prediksi model terhadap nilai rating aktual. Nilai RMSE lebih kecil berarti lebih baik.

In [112]:
from surprise import accuracy

rmse_score = accuracy.rmse(predictions)

RMSE: 1.6399


Metrik Tambahan: Precision@K dan Recall@K

Metrik ini digunakan untuk mengevaluasi seberapa relevan hasil rekomendasi bagi user. Kita gunakan K = 5.

In [113]:
from collections import defaultdict

def precision_recall_at_k(predictions, k=5, threshold=7):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = {}
    recalls = {}

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        top_k = user_ratings[:k]

        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in top_k)
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in top_k)

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
    avg_recall = sum(rec for rec in recalls.values()) / len(recalls)

    return avg_precision, avg_recall

In [114]:
precision, recall = precision_recall_at_k(predictions, k=5)
print(f"Precision@5: {precision:.4f}")
print(f"Recall@5: {recall:.4f}")

Precision@5: 0.7062
Recall@5: 0.7025
